In [1]:
%matplotlib inline

from time import time
from math import sqrt, floor
import numpy as np
import pandas as pd
from IPython.core.debugger import set_trace

pd.options.display.float_format = '{:.3f}'.format

import matplotlib.pyplot as plt
plt.style.use = "default"

In [2]:
def merge(df, labels):
    return labels.merge(df, left_index=True,right_index=True)

In [3]:
train = pd.read_csv("TrainData2.txt", delim_whitespace=True, header=None)
labels = pd.read_csv("TrainLabel2.txt", header=None)
labels = labels.rename(columns={0: "label"})

In [4]:
train.describe()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
count,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000,...,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000
mean,58.022,38.382,61.468,96.175,58.098,99.197,85.865,27.375,58.880,55.787,...,-3.341,-20.005,-1.086,-4.376,-21.664,-0.980,-4.633,-18.996,-1.702,-4.229
std,11.705,14.590,15.003,11.120,10.655,9.439,16.643,7.445,8.885,12.347,...,8.921,2.908,1.055,1.851,3.595,0.308,1.054,3.307,1.018,1.150
min,31.000,23.000,47.000,69.000,43.000,83.000,42.000,19.000,45.000,4.950,...,-58.280,-26.790,-5.510,-10.120,-34.630,-1.830,-7.970,-29.340,-6.500,-8.930
25%,50.000,28.000,52.000,89.000,51.000,93.000,73.000,24.000,54.000,48.370,...,-4.660,-22.250,-1.750,-5.530,-24.220,-1.190,-5.410,-21.780,-2.360,-4.870
50%,57.000,32.000,55.000,95.000,54.000,96.000,85.000,25.000,57.000,57.560,...,-1.250,-19.950,-1.030,-4.490,-21.040,-0.990,-4.670,-18.870,-1.650,-4.150
75%,65.000,43.000,65.000,103.000,63.000,103.000,98.000,27.000,60.000,64.120,...,1.430,-18.250,-0.390,-2.770,-19.060,-0.780,-3.900,-16.770,-1.030,-3.290
max,107.000,91.000,124.000,141.000,100.000,138.000,136.000,84.000,114.000,86.080,...,9.580,-7.760,1.780,1.040,-12.070,0.260,-0.770,-8.330,2.580,-0.590


In [5]:
labels.describe()

,label
count,325.000
mean,2.077
std,1.002
min,1.000
25%,1.000
50%,2.000
75%,3.000
max,4.000


### Data Cleaning
Let's make it easier to deal with this data by making outliers into averages. We will take all values higher or lower than 3\*IQR and will replace them with the mean of that column

In [6]:
for col_name in train.columns:
    train_col = train[col_name]
    
    q1 = train_col.quantile(0.25)
    q3 = train_col.quantile(0.75)
    iqr = q3 - q1
    
    outside_iqr = (train_col < (q1 - 3 * iqr)) | (train_col > (q3 + 3 * iqr))
    new_col_mean = train_col[~outside_iqr].mean()
    
    train[col_name][outside_iqr] = new_col_mean

/Users/luis/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/luis/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/Users/luis/.pyenv/versions/3.4.3/lib/python3.4/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#inde

In [7]:
train.describe()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,25,26
count,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000,...,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000,325.000
mean,58.022,38.059,60.032,96.175,57.969,99.077,85.865,25.383,57.538,55.787,...,-1.930,-20.005,-1.086,-4.376,-21.664,-0.980,-4.633,-18.996,-1.687,-4.229
std,11.705,13.999,12.297,11.120,10.397,9.189,16.643,2.406,5.522,12.347,...,5.446,2.908,1.055,1.851,3.595,0.308,1.054,3.307,0.982,1.150
min,31.000,23.000,47.000,69.000,43.000,83.000,42.000,19.000,45.000,4.950,...,-22.780,-26.790,-5.510,-10.120,-34.630,-1.830,-7.970,-29.340,-6.110,-8.930
25%,50.000,28.000,52.000,89.000,51.000,93.000,73.000,24.000,54.000,48.370,...,-3.570,-22.250,-1.750,-5.530,-24.220,-1.190,-5.410,-21.780,-2.360,-4.870
50%,57.000,32.000,55.000,95.000,54.000,96.000,85.000,25.000,57.000,57.560,...,-1.250,-19.950,-1.030,-4.490,-21.040,-0.990,-4.670,-18.870,-1.650,-4.150
75%,65.000,43.000,64.000,103.000,62.000,103.000,98.000,26.000,59.000,64.120,...,1.430,-18.250,-0.390,-2.770,-19.060,-0.780,-3.900,-16.770,-1.030,-3.290
max,107.000,84.000,103.000,141.000,94.000,131.000,136.000,36.000,78.000,86.080,...,9.580,-7.760,1.780,1.040,-12.070,0.260,-0.770,-8.330,2.580,-0.590


### Correlation

In [8]:
def corr(labels, train):
    n = len(labels)
    v1, v2 = labels.values, train.values
    sums = np.multiply.outer(v2.sum(0), v1.sum(0))
    stds = np.multiply.outer(v2.std(0), v1.std(0))
    return pd.DataFrame((v2.T.dot(v1) - sums / n) / stds / n, train.columns, labels.columns)

def count_corrs(correlation_result, with_labels=True):
    if with_labels:
        return [(i, correlation_result[correlation_result > i].dropna().size) for i in np.arange(0,1.1,.1)]
    else:
        return [correlation_result[correlation_result > i].dropna().size for i in np.arange(0,1.1,.1)]

In [9]:
train_label_corr = corr(labels, train)
count_corrs(train_label_corr)

[(0.0, 18),
 (0.10000000000000001, 9),
 (0.20000000000000001, 6),
 (0.30000000000000004, 3),
 (0.40000000000000002, 1),
 (0.5, 0),
 (0.60000000000000009, 0),
 (0.70000000000000007, 0),
 (0.80000000000000004, 0),
 (0.90000000000000002, 0),
 (1.0, 0)]

### Splitting

In [10]:
from sklearn.model_selection import train_test_split
X, y = train, labels.iloc[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

### Random Forest Classifier
Below we try out the Random Forest Classifier provided by ScikitLearn to see how well it performs with this dataset

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [12]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.78787878787878785

In [13]:
best_depth = 2
best_score = 0
for current_depth in range(2, 100):
        clf = RandomForestClassifier(max_depth=current_depth, random_state=0)
        clf.fit(X_train, y_train)
        current_score = clf.score(X_test, y_test)

        if current_score > best_score:
            best_score = current_score
            best_depth = current_depth
        
print('Best depth was {} with a score of {}'.format(best_depth, best_score))

Best depth was 10 with a score of 0.8484848484848485


### Results
As we can see, given the 90/10 split with a `random_state = 0`, the best accuracy we got from the **Random Forest Classifier** was **75.76%** with a `max_depth = 5` and `random_state = 0`

In [14]:
from sklearn.model_selection import cross_val_score

In [15]:
clf = RandomForestClassifier(max_depth=5, random_state=0)
scores = cross_val_score(clf, X, y)
scores.mean()

0.83405026380104264

In [16]:
var = train.var().sort_values(ascending=False)
train_label_var = (var - var.min()) / (var.max() - var.min()) # Normailizing variance

In [17]:
best_corr = 0
best_score = 0
best_var = 0

for i in range(10):
    curr_corr = .1 * i
    corr_table = train_label_corr[train_label_corr >= curr_corr].dropna()
    
    if corr_table.empty:
        break
    
    for j in range(10):
        curr_var = .1 * j
        var_table = train_label_var[train_label_var >= curr_var]

        if var_table.empty:
            break

        curr_train = train[list(set(var_table.index.append(corr_table.index)))]
        X, y = curr_train, labels.iloc[:,0]

        clf = RandomForestClassifier(max_depth=5, random_state=0)
        scores = cross_val_score(clf, X, y)
        curr_score = scores.mean()

        if curr_score > best_score:
            best_score = curr_score
            best_corr = curr_corr
            best_var = curr_var
        
print('Best correlation {} with best variance {} yield best score {}'.format(best_corr, best_var, best_score))

Best correlation 0.0 with best variance 0.0 yield best score 0.8340502638010426


### Conclusion

By choosing only columns with a correlation of `0.2` or more or a variance of `0.6` or more, we can increase our accuracy score from `82.09%` to **`83.01%`**.

A `0.92%` increase is not significant enough and could be attributed to randomness, so the smartest choice would probably be to keep all columns.

In [19]:
train.to_csv("cleaned_train.csv")
labels.to_csv("cleaned_label.csv")

In [20]:
test = pd.read_csv("TestData2.txt", delim_whitespace=True, header=None)
test.to_csv('cleaned_test.csv')